# 프로세스와 스레드

## 프로세스의 종류
- 포그라운드 프로세스: 사용자가 볼 수 있는 곳에서 실행되는 프로세스
- 백그라운드 프로세스: 사용자가 보지 못하는 곳에서 실행되는 프로세스
  - 데몬/서비스 프로세스: 주어진 작업만을 수행하는 백그라운드 프로세스

## 사용자 영역에서의 메모리 배치
![](https://lh4.googleusercontent.com/proxy/TRLpOmIXbQEr9WH4By3hr__2cBYkXKRqE12QwQEQg9Hi0yrqFp93-e7ZlGSgErVztnre717GKI7KC_x1V_gQ1QHd--dw2JjcADVnSaVGgw)

- 코드 영역: 실행 가능한 명령어가 저장되는 공간
- 데이터 영역: 프로그램이 실행되는 동안 유지할 데이터가 저장되는 공간 (정적변수나 전역변수가 저장)
- 힙 영역: 개발자가 직접 할당 가능한 저장 공간 (동적 메모리 할당)
  - 메모리 누수(memory leak): 할당된 메모리를 해제하지 않아 사용하지 않는 메모리가 계속 남아있는 현상
  - 가비지 컬렉션(garbage collection): 사용하지 않는 메모리를 자동으로 해제하는 기능
- 스택 영역: 함수 호출 시 생성되는 지역 변수, 매개 변수, 함수 복귀 주소 등이 저장되는 공간
  - 스택 트레이스: 함수 호출의 역순으로 쌓인 함수들의 목록

### Why? 사용자 영역에서의 메모리 배치를 나눈 것인가?
- 효율성 → 코드, 전역 변수, 힙, 스택을 분리해 관리 효율 극대화
- 안정성 → 잘못된 메모리 접근으로부터 다른 영역 보호
- 보안성 → 커널 영역과 사용자 영역을 분리하여 시스템 보호
- 프로세스 독립성 → 프로세스 간 메모리 간섭 방지
- 참고자료: https://www.geeksforgeeks.org/c/memory-layout-of-c-program/

## PCB와 문맥 교환
- PCB(Process Control Block): 프로세스의 상태와 정보를 저장하는 자료구조
  - 프로세스 ID, 프로그램 카운터, CPU 레지스터, 메모리 관리 정보, 입출력 상태 정보 등
  - 운영체제가 프로세스를 관리하는 데 필요한 모든 정보 포함
  - PCB 구조체의 예시
  ![](https://postfiles.pstatic.net/20160429_171/eldkrpdla121_14619047144741Sxu1_PNG/1.png?type=w2)
  - PCB들은 프로세스 테이블의 형태로 저장되기도 함
  ![](https://csnote.net/assets/img/os/process_table.png)
- 문맥 교환: 기존 프로세스의 문맥을 PCB에 백업하고, 새로운 프로세스 문맥을 PCB로부터 복구하여 새로운 프로세스를 실행하는 과정
  - 문맥 교환 과정
  ![](https://csnote.net/assets/img/os/context_switch.png)
  - 타이머 인터럽트: 프로세스의 CPU 사용 시간을 제한하기 위해 설정된 타이머가 만료되었을 때 발생하는 인터럽트

### Why? PCB와 문맥 교환이 필요한가?
| 항목           | 필요 이유                | 결과                    |
| ------------ | -------------------- | --------------------- |
| **PCB**      | 프로세스 상태 저장 및 관리      | 멀티프로세스 환경에서 안정성 보장    |
| **문맥 교환**    | 여러 프로세스가 CPU를 번갈아 사용 | 멀티태스킹 및 자원 활용 최적화     |
| **두 개념의 관계** | PCB를 통해 문맥 교환 가능     | PCB 없이는 프로세스 상태 복원 불가 |

## 프로세스의 상태
![](https://csnote.net/assets/img/os/process_state.png)
- 생성 상태 (new)
  - 프로세스를 생성 중인 상태
  - 메모리에 적재되어 PCB를 할당받은 상태
  - 생성 상태를 거져 실행할 준비가 완료된 프로세스는 준비 상태가 되어 CPU의 할당을 기다림
- 준비 상태 (ready)
  - CPU 할당 받을 수 있지만, 차례를 기다리는 상태
  - 준비 상태인 프로세스가 CPU를 할당받으면 실행 상태가 됨
  - **디스패치**: 준비 상태에서 실행 상태로 전환하는 과정
- 실행 상태 (running)
  - CPU를 할당받아 명령어를 실행하는 상태
  - 일정 시간 동안만 CPU를 사용할 수 있음
  - 할당된 시간이 끝나, 타이머 인터럽트가 발생하면 다시 준비 상태가 됨
  - 실행 도중 입출력장치를 사용하여 입출력장치의 작업이 끝날 때까지 기다려야 하면 대기 상태가 됨
- 대기 상태 (blocked)
  - 바로 실행이 불가능한 조건에 놓이는 경우의 상태
    - 프로세스가 입출력 작업을 요청
    - 바로 확보할 수 없는 자원을 요청
  - 실행 가능한 상태가 되면 다시 준비 상태가 되어 CPU 할당을 기다림
- 종료 상태 (terminated)
  - 프로세스가 종료된 상태를 의미
  - 운영체제는 PCB와 프로세스가 사용한 메모리를 정리함

### 블로킹 입출력과 논블로킹 입출력
- 블로킹 입출력 (blocking I/O)
  - ![](https://csnote.net/assets/img/os/blocking_io.png)
  - 입출력 작업이 완료될 때까지 프로세스가 대기 상태에 머무르는 방식
  - CPU 자원을 효율적으로 사용하지 못할 수 있음
- 논블로킹 입출력 (non-blocking I/O)
  - ![](https://csnote.net/assets/img/os/nonblocking_io.png)
  - 입출력 작업이 완료되지 않아도 프로세스가 계속 실행될 수 있는 방식
  - CPU 자원을 더 효율적으로 사용할 수 있음
- 블로킹, 논블로킹 입출력의 비교

| 항목        | 블로킹 입출력                    | 논블로킹 입출력                 |
| ----------- | ------------------------------- | ------------------------------- |
| **정의**    | 입출력 작업 완료까지 대기          | 입출력 작업 완료 여부와 상관없이 실행 계속 |
| **장점**    | 구현이 간단하고 직관적            | CPU 자원 활용 극대화             |
| **단점**    | CPU 자원 낭비 가능성              | 구현 복잡성 증가                 |
| **사용 사례** | 단순한 입출력 작업에 적합         | 고성능 서버, 실시간 시스템에 적합    |



# 멀티프로세스와 멀티스레드
- 스레드: 프로세스를 구성하는 실행 흐름 단위
- ![](https://csnote.net/assets/img/os/thread.png)
- 멀티프로세스: 여러 개의 프로세스를 동시에 실행하는 실행 방식
- 멀티스레드: 여러 개의 스레드로 프로세스를 동시에 실행하는 실행 방식
- 멀티프로세스와 멀티스레드의 비교
| 항목          | 멀티프로세스                      | 멀티스레드                     |
|--------- | ------------------------------- | ------------------------------- |
| **정의**      | 여러 개의 프로세스를 동시에 실행하는 방식 | 하나의 프로세스 내에서 여러 스레드를 동시에 실행하는 방식 |
| **메모리 사용** | 각 프로세스가 독립된 메모리 공간을 가짐    | 스레드들이 같은 메모리 공간을 공유함    |
| **오버헤드**   | 프로세스 간 전환 비용이 큼           | 스레드 간 전환 비용이 적음          |
| **안정성**    | 프로세스 간 메모리 간섭 없음          | 스레드 간 메모리 간섭 가능성 있음   |
| **통신**      | 프로세스 간 통신(IPC)이 필요함       | 스레드 간 통신이 용이함            |
| **사용 사례** | 독립적인 작업 단위가 필요한 경우        | 경량화된 작업 단위가 필요한 경우       |
- 스레드 조인: 하나의 스레드가 다른 스레드가 종료될 때까지 기다리는 기능 -> 비동기적 프로그래밍과 유사

In [13]:
import multiprocessing, threading, os

def one():
    pid = os.getpid()
    tid = threading.get_native_id()
    print(f"one: PID={pid} tid={tid}", flush=True)

def two():
    pid = os.getpid()
    tid = threading.get_native_id()
    print(f"two: PID={pid} tid={tid}", flush=True)

if __name__ == '__main__':
    p1 = multiprocessing.Process(target=one)
    p2 = multiprocessing.Process(target=two)

    p1.start()
    p2.start()

    p1.join()
    p2.join()

In [2]:
# 함수 foo, bar, baz를 각각의 스레드에서 실행
# 각 함수 내에서 프로세스 ID와 스레드 ID를 출력

import threading
import os


def foo():
    pid = os.getpid()  # 현재 프로세스의 pid를 반환
    tid = threading.get_native_id()  # 현재 스레드의 id를 반환
    print(f"foo: PID={pid}, Thread ID={tid}")  # pid와 tid 값을 출력


def bar():
    pid = os.getpid()
    tid = threading.get_native_id()
    print(f"bar: PID={pid}, Thread ID={tid}")


def baz():
    pid = os.getpid()
    tid = threading.get_native_id()
    print(f"baz: PID={pid}, Thread ID={tid}")


if __name__ == "__main__":
    thread1 = threading.Thread(target=foo)  # 첫 번째 스레드 생성, 실행할 함수는 foo
    thread2 = threading.Thread(target=bar)  # 두 번째 스레드 생성, 실행할 함수는 bar
    thread3 = threading.Thread(target=baz)  # 세 번째 스레드 생성, 실행할 함수는 baz
    thread1.start()  # 첫 번째 스레드 실행
    thread2.start()  # 두 번째 스레드 실행
    thread3.start()  # 세 번째 스레드 실행

foo: PID=31908, Thread ID=13160
bar: PID=31908, Thread ID=30396
baz: PID=31908, Thread ID=8076


# 프로세스간 통신
- 프로세스 간 데이터 공유 방법
    - 공유 메모리: 프로세스가 공유할 수 있는 메모리 영역
    - 프로세스 간 통신: 프로세스 간에 주고 받을 데이터를 메시지 형태로 주고 받는 방식

## 공유 메모리
- ![](https://csnote.net/assets/img/os/shared_memory.png)
- 특정 메모리 공간에 메모리를 읽고 쓰며, 데이터를 공유하는 방식
- 각 프로세스가 자신의 메모리 영역을 쓰고 읽는 것처럼 통신
- 커널의 개입이 없어, 통신 속도가 빠름
- 동시에 공유 메모리를 사용할 경우, 데이터의 일관성이 깨질 수 있음 (레이스 컨디션)

## 메시지 전달
- ![](https://csnote.net/assets/img/os/message_passing.png)
- 프로세스 간에 주고받을 데이터가 커널을 거쳐 송수신 되는 방식
- 커널의 개입이 있어, 레이스 컨디션이나, 동기화 등의 문제는 없음
- 공유 메모리 방식 보다는 속도가 느림
- 파이프, 시그널, 소켓, 원격 프로시저 호출(RPC) 등이 존재

### 파이프
- ![](https://csnote.net/assets/img/os/unnamed_pipe.png)
- 프로세스 간의 단방향 통신 도구
- 익명 파이프(Unnamed Pipe): 전통적인 파이프 방식으로, 부모-자식 간 통신만 가능
- 지명 파이프(Named Pipe): 양방향 통신을 지원하며, 타 프로세스간 통신까지 가능

### 시그널
- 프로세스에게 특정 이벤트가 발생했음을 알리는 비동기적인 신호
- 시그널 핸들러를 이용해, 시그널을 처리하게 됨
    - 프로세스는 특정 시그널을 발생시킬 수 있음
    - 시그널 핸들러를 재 정의할 수 있음
- 코어 덤프: 프로세스가 비정상적으로 종료될 때, 프로세스의 메모리 정보를 파일로 저장하는 기능
    - 하단 코드가 의도적으로 에러를 발생시켜, 코어 덤프를 만드는 코드입니다만, 여기서는 생성되지 않는 문제가 있습니다.

### RPC나 네트워크 소켓
- RPC: 원격 코드를 실행하는 기숳 / 다른 프로세스의 프로시져를 호출할 수 있음
    - 성능 저하를 최소화하고, 메시지 송수신이 가능함
    - 서버 간 통신에서 많이 사용되는 경우가 많음
- 네트워크 소켓: 5장에서 소개할 예정

In [3]:
import ctypes

def bug():
    ctypes.string_at(0)

bug()

OSError: exception: access violation reading 0x0000000000000000

# 참고자료
- https://csnote.net/
- https://blog.naver.com/eldkrpdla121/220696668120
- https://www.tcpschool.com/c/c_memory_structure
- https://github.com/kangtegong/cs/blob/main/os/